## Merge Circle chart Data

매주 둘째주 목요일 오전 10시에 써클차트(https://circlechart.kr/)에 음반판매량이 업데이트 됨

써클차트는 국내 및 글로벌 음악 서비스 플랫폼의 K-pop 데이터를 정식 공급받는 국내 음악차트이며, 써클차트의 월간 음반 판매량은 연예기획사들의 매달 실적을 추정할 수 있는 좋은 지표입니다.

In [32]:
CHART_PATH = '../data/global_kpop_chart.csv'
ALBUM_PATH = '../data/album_chart.csv'
OUTPUT_PATH = '../data/global_kpop_chart_cleanup3.xlsx'

In [33]:
import pandas as pd
import numpy as np

In [34]:
global_chart = pd.read_csv(CHART_PATH, usecols=['month', 'artist', 'producer', 'album', 'title'])
global_chart = global_chart.astype({
    'month': 'int32',
    'artist': 'string',
    'producer': 'string',
    'album': 'string',
    'title': 'string'
}, errors='raise')
global_chart['artist'] = global_chart['artist'].str.split(pat='|', n=1).str[0]
global_chart = global_chart.drop_duplicates(subset=['artist', 'producer'], keep='first')
global_chart = global_chart.astype({'artist': 'string'}, errors='raise')

In [35]:
global_chart.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 206 entries, 0 to 1933
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   month     206 non-null    int32 
 1   album     206 non-null    string
 2   artist    206 non-null    string
 3   producer  206 non-null    string
 4   title     206 non-null    string
dtypes: int32(1), string(4)
memory usage: 8.9 KB


In [36]:
album_chart = pd.read_csv(ALBUM_PATH, usecols=['month', 'artist', 'sales_volume', 'album'])
album_chart = album_chart.astype({'month': 'int32', 'artist': 'string', 'album': 'string'}, errors='raise')
album_chart[['monthly_sales', 'annual_sales']] = album_chart['sales_volume'].str.split(pat='/', n=1, expand=True)
album_chart = album_chart.astype({'monthly_sales': 'int32', 'annual_sales': 'int32'}, errors='raise')
album_chart = album_chart.drop(columns=['sales_volume']).drop_duplicates(['artist', 'month'])
# Caution: Some artists has multiple agencies that has changed
album_chart = album_chart.reindex(columns=['month', 'album', 'artist', 'monthly_sales', 'annual_sales'])
album_chart = album_chart.sort_values(by=['month', 'monthly_sales'])

In [37]:
album_chart.head(20)

,month,album,artist,monthly_sales,annual_sales
98,1202,1집 Heartbreaker,G-Dragon,832,1382
92,1202,1집 열꽃,타블로 (에픽하이),915,2775
91,1202,GD & TOP,"GD, TOP (빅뱅)",943,2543
88,1202,1 (Remastered),The Beatles,953,1577
85,1202,We Sing. We Dance. We Steal Things.,Jason Mraz,1014,1795
80,1202,Guilt-Free,이이언 (eAeon),1064,1806
76,1202,Up All Night,One Direction,1080,1080
74,1202,Mozart L'Opera Rock (뮤지컬 모짜르트 락 오페라) OST (Fran...,Mozart Opera Rock,1099,1250
71,1202,Hands Up,2PM,1166,3287
70,1202,1집 So Cool,씨스타 (Sistar),1187,2342


In [38]:
album_chart.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7709 entries, 98 to 12900
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   month          7709 non-null   int32 
 1   album          7709 non-null   string
 2   artist         7709 non-null   string
 3   monthly_sales  7709 non-null   int32 
 4   annual_sales   7709 non-null   int32 
dtypes: int32(3), string(2)
memory usage: 271.0 KB


In [39]:
# Search producer from producer dataframe and insert into new_sales
new_sales = pd.merge(left=album_chart, right=global_chart, how='outer', on='artist')
new_sales['month_y'] = new_sales['month_y'].fillna(1800)
new_sales['producer'] = new_sales['producer'].fillna('미상')
new_sales = new_sales.astype({'month_y': 'int32', 'artist': 'string', 'producer': 'string'}, errors='raise')
# find the producer artist belonged to before the release of the album
new_sales = new_sales.sort_values('month_y', ascending=True).drop_duplicates(['artist', 'month_x'])
new_sales.rename(columns={'month_x': 'month', 'album_x': 'album'}, inplace=True)
new_sales = new_sales.drop(columns=['month_y', 'album_y', 'title'])
new_sales = new_sales.reindex(columns=['month', 'album', 'artist', 'producer', 'monthly_sales', 'annual_sales'])
new_sales = new_sales.sort_values(by=['month', 'monthly_sales'])
new_sales = new_sales.dropna(axis="index")
new_sales = new_sales.astype({'month': 'int32', 'annual_sales': 'int32', 'monthly_sales': 'int32'}, errors='raise')

In [40]:
new_sales.head(100)

,month,album,artist,producer,monthly_sales,annual_sales
0,1202,1집 Heartbreaker,G-Dragon,미상,832,1382
39,1202,1집 열꽃,타블로 (에픽하이),미상,915,2775
41,1202,GD & TOP,"GD, TOP (빅뱅)",미상,943,2543
61,1202,1 (Remastered),The Beatles,미상,953,1577
66,1202,We Sing. We Dance. We Steal Things.,Jason Mraz,미상,1014,1795
...,...,...,...,...,...,...
1566,1203,Get your Swag On (Single Album),토니안 & 스매쉬 (Smash),미상,5732,5732
1281,1203,Warrior,B.A.P,미상,5888,23821
1567,1203,Gift : E.C.H.O (Special Album),박효신,미상,6241,6241
1583,1203,Face (Single),뉴이스트 (NU`EST),미상,6462,6462


In [41]:
new_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7709 entries, 0 to 6879
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   month          7709 non-null   int32 
 1   album          7709 non-null   string
 2   artist         7709 non-null   string
 3   producer       7709 non-null   string
 4   monthly_sales  7709 non-null   int32 
 5   annual_sales   7709 non-null   int32 
dtypes: int32(3), string(3)
memory usage: 331.2 KB


In [42]:
sales_table = pd.pivot_table(
    new_sales,
    values="monthly_sales",
    index=['producer', 'artist', 'album'],
    columns="month",
    aggfunc=np.sum
).fillna(0)

In [48]:
sales_table.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3758 entries, ('ADOR', 'NewJeans', '1st EP `New Jeans`') to ('하이업엔터테인먼트', 'STAYC(스테이씨)', 'YOUNG-LUV.COM')
Columns: 127 entries, 1202 to 2209
dtypes: float64(127)
memory usage: 3.8+ MB


In [43]:
sales_table.head(20)

month                                                                                          1202  \
producer                              artist                  album                                   
ADOR                                  NewJeans                1st EP `New Jeans`                0.0   
                                                              1st EP `New Jeans` (Weverse)      0.0   
AMBITION MUSIK, Daytona Entertainment 릴러말즈 (Leellamarz), TOIL TOYSTORY3                         0.0   
AOMG                                  박재범                     1집 New Breed                  73841.0   
                                                              EVERYTHING YOU WANTED             0.0   
...                                                                                             ...   
JYP Entertainment                     TWICE                   Taste of Love                     0.0   
                                                              The year of `YES`                 0.0   
                                                              What is Love?                     0.0   
                                                              YES or YES                        0.0   
                                                              twicetagram                       0.0   

month                                                                                       1203  \
producer                              artist                  album                                
ADOR                                  NewJeans                1st EP `New Jeans`             0.0   
                                                              1st EP `New Jeans` (Weverse)   0.0   
AMBITION MUSIK, Daytona Entertainment 릴러말즈 (Leellamarz), TOIL TOYSTORY3                      0.0   
AOMG                                  박재범                     1집 New Breed                   0.0   
                                                              EVERYTHING YOU WANTED          0.0   
...                                                                                          ...   
JYP Entertainment                     TWICE                   Taste of Love                  0.0   
                                                              The year of `YES`              0.0   
                                                              What is Love?                  0.0   
                                                              YES or YES                     0.0   
                                                              twicetagram                    0.0   

month                                                                                       1204  \
producer                              artist                  album                                
ADOR                                  NewJeans                1st EP `New Jeans`             0.0   
                                                              1st EP `New Jeans` (Weverse)   0.0   
AMBITION MUSIK, Daytona Entertainment 릴러말즈 (Leellamarz), TOIL TOYSTORY3                      0.0   
AOMG                                  박재범                     1집 New Breed                   0.0   
                                                              EVERYTHING YOU WANTED          0.0   
...                                                                                          ...   
JYP Entertainment                     TWICE                   Taste of Love                  0.0   
                                                              The year of `YES`              0.0   
                                                              What is Love?                  0.0   
                                                              YES or YES                     0.0   
                                                              twicetagram                    0.0   

month                                                     

In [44]:
data_frames = {
    'cleanup': sales_table,
    'sales_with_producer': new_sales,
    'raw_sales': global_chart,
    'raw_producer': album_chart,
}
# Create a Pandas Excel writer using XlsxWriter as the engine.
with pd.ExcelWriter(OUTPUT_PATH, engine='xlsxwriter') as writer:
    for sheet_name, data_frame in data_frames.items():
        data_frame.to_excel(writer, sheet_name=sheet_name)